<a href="https://colab.research.google.com/github/JuliethQP/modelos_ia_prueba/blob/master/modelo_tumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importar librerías necesarias
from google.colab import files
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input
import pandas as pd
import zipfile
import os

# Importar librerías necesarias

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam



uploaded = files.upload()

with zipfile.ZipFile('dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

train_dir = './dataset/'

# Prepocesar, aumentar y dividir el conjunto de datos
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Leer las imagenes del directorio
train_generator = train_datagen.flow_from_directory(
    train_dir, # directorio de entrenamiento
    target_size=(224, 224), # redimensionar las imágenes
    batch_size=32, # tamaño del lote
    class_mode='categorical', # usar clasificación categórica
    subset='training',  # dividir en conjunto de entrenamiento
    shuffle=True # mezclar los datos
)

# Construir conjunto dde validación
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

Saving dataset.zip to dataset.zip
Found 4847 images belonging to 3 classes.
Found 1209 images belonging to 3 classes.


In [ ]:
## Cargar el modelo

from tensorflow.keras.applications import VGG16
from tensorflow.keras import Input

# Cargar VGG16 sin las capas de clasificación (include_top=False)
base_model = VGG16(
    weights='imagenet',    # Pesos preentrenados en ImageNet
    include_top=False,     # No incluir las capas fully connected del final
    input_tensor=Input(shape=(224, 224, 3))  # Tamaño esperado de entrada
)

# Congelar las capas del modelo base para que no se entrenen
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
## Añadir nuevas capas de clasificación

from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model

x = base_model.output
x = Flatten()(x)                # Aplanar los mapas de activación
x = Dense(256, activation='relu')(x)  # Capa totalmente conectada
x = Dropout(0.5)(x)             # Dropout para evitar overfitting
predictions = Dense(3, activation='softmax')(x)  # 3 clases → softmax

# Crear el modelo final
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
## Compilar el modelo
from tensorflow.keras.optimizers import Adam
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=10
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_23']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


151/151 ━━━━━━━━━━━━━━━━━━━━ 104s 602ms/step - accuracy: 0.6562 - loss: 0.7605 - val_accuracy: 0.8775 - val_loss: 0.3531
Epoch 2/10
  1/151 ━━━━━━━━━━━━━━━━━━━━ 23s 155ms/step - accuracy: 0.9375 - loss: 0.3124

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


151/151 ━━━━━━━━━━━━━━━━━━━━ 21s 138ms/step - accuracy: 0.9375 - loss: 0.3124 - val_accuracy: 0.8657 - val_loss: 0.3598
Epoch 3/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 100s 542ms/step - accuracy: 0.8553 - loss: 0.3654 - val_accuracy: 0.8682 - val_loss: 0.3659
Epoch 4/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 21s 141ms/step - accuracy: 0.8438 - loss: 0.4287 - val_accuracy: 0.8708 - val_loss: 0.3606
Epoch 5/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 121s 546ms/step - accuracy: 0.8764 - loss: 0.3207 - val_accuracy: 0.8809 - val_loss: 0.3043
Epoch 6/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 16s 106ms/step - accuracy: 0.9375 - loss: 0.2251 - val_accuracy: 0.8750 - val_loss: 0.3213
Epoch 7/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 126s 547ms/step - accuracy: 0.8744 - loss: 0.3032 - val_accuracy: 0.9113 - val_loss: 0.2574
Epoch 8/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 17s 110ms/step - accuracy: 0.9062 - loss: 0.1968 - val_accuracy: 0.8986 - val_loss: 0.2652
Epoch 9/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 124s 539ms/step - accuracy: 0.8982 - loss: 0.2726 -

In [ ]:
model.save("modelo_tumores_vgg16.h5")

Validacion del modelo

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import requests

import requests
from PIL import Image
from io import BytesIO
from tensorflow.keras.preprocessing import image

# Cargar modelo
url = "https://huggingface.co/JuliethQP/modelo_tumores/resolve/main/modelo_tumores_vgg16.h5"
with open("modelo.h5", "wb") as f:
    f.write(requests.get(url).content)
model = load_model("modelo.h5")

# Definir clases (ajusta esto según tu modelo)
clases = ['glioma', 'meningioma', 'pituitary']

# Cargar imagen de prueba
url = "https://huggingface.co/JuliethQP/modelo_tumores/resolve/main/imagenes_test/brain_g.jpg"

# Descargar la imagen
response = requests.get(url)
img = Image.open(BytesIO(response.content)).resize((224, 224))

img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predecir
predicciones = model.predict(img_array)
indice_predicho = np.argmax(predicciones)
clase_predicha = clases[indice_predicho]
confianza = predicciones[0][indice_predicho]

# Mostrar resultado
print(f"Tipo de cáncer predicho: {clase_predicha} (confianza: {confianza:.2%})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Tipo de cáncer predicho: glioma (confianza: 99.72%)
